###Import libraries

In [1]:
!pip install streamlit
!pip install pyngrok===4.1.1
!pip install plotly
!pip install icd10-cm
!pip install scikit-learn===24.2
!pip install bz2file
from pyngrok import ngrok

     |████████████████████████████████| 8.3 MB 2.3 MB/s 
     |████████████████████████████████| 75 kB 3.7 MB/s 
     |████████████████████████████████| 180 kB 25.0 MB/s 
     |████████████████████████████████| 111 kB 39.2 MB/s 
     |████████████████████████████████| 4.3 MB 36.9 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 124 kB 49.6 MB/s 
     |████████████████████████████████| 788 kB 30.6 MB/s 
     |████████████████████████████████| 370 kB 45.8 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=75596a7a9897052efe0f7e6a8630099d0be2e3e93874411b784fe2fa0c54cf8c
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting

  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15984 sha256=6fffa96dca8336aa41bd068886adc5c923a1488e4e6aaf048fb31baed1d7895d
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok
     |████████████████████████████████| 675 kB 2.2 MB/s 
ERROR: Could not find a version that satisfies the requirement scikit-learn===24.2 (from versions: 0.9, 0.10, 0.11, 0.12, 0.12.1, 0.13, 0.13.1, 0.14, 0.14.1, 0.15.0b1, 0.15.0b2, 0.15.0, 0.15.1, 0.15.2, 0.16b1, 0.16.0, 0.16.1, 0.17b1, 0.17, 0.17.1, 0.18, 0.18.1, 0.18.2, 0.19b2, 0.19.0, 0.19.1, 0.19.2, 0.20rc1, 0.20.0, 0.20.1, 0.20.2, 0.20.3, 0.20.4, 0.21rc2, 0.21.0, 0.21.1, 0.21.2, 0.21.3, 0.22rc2.post1, 0.22rc3, 0.22, 0.22.1, 0.22.2, 0.22.2.post1, 0.23.0rc1, 0.23.0, 0.23.1, 0.23.2, 0.24.dev0, 0.24.0rc1, 0.24.0, 0.24.1, 0.24.2, 1.0rc1, 1.0rc2, 1.0)
ERROR: No matching distribution found for scikit-learn===24.2
  Created wheel for bz2file: filename=bz2

###Pickle model

In [2]:
# Save Model Using Pickle
import pandas as pd
from sklearn import model_selection
from sklearn.ensemble import RandomForestRegressor
import bz2file as bz2
import pickle

def compressed_pickle(title, data):
  with bz2.BZ2File(title + '.pbz2', 'w') as f:
    pickle.dump(data, f)

df = pd.read_excel('/content/alldischarges_new.xlsx')
df.dropna(inplace=True)
X = df[['Years','District_code','dSex','Age','Chapter']]
print(X)
y = df[['meanStay']]
# Split the data set in a training set (70%) and a test set (30%)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.30)

model = RandomForestRegressor()

# Train the model
model.fit(X_train, y_train)

        Years  District_code  dSex  Age  Chapter
0        2014             11     0   15        1
1        2014              1     0   15        1
2        2014             16     1   20        1
3        2014              1     0   25        1
4        2014              3     0   25        1
...       ...            ...   ...  ...      ...
129338   2018              3     0   55       21
129339   2018             12     0   55       21
129340   2018              3     1   60       21
129341   2018             10     0   60       21
129342   2018             10     1   65       21

[129343 rows x 5 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [4]:
# Save the trained model to a file so we can use it to make predictions later
compressed_pickle('randforestreg', model.fit(X_train, y_train))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


###Web App

In [9]:
%%writefile NZ_app.py
import streamlit as st
import icd10
import sklearn
import pandas as pd
import numpy as np
import pickle
import bz2file as bz2
from datetime import datetime, timedelta, date
import datetime as dt
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import LabelEncoder

#decompress
def decompress_pickle(file):
  data = bz2.BZ2File(file, 'rb')
  data = pickle.load(data)
  return data  
st.set_page_config(page_title="NZ Hospital Demand Forecast APP",layout="centered",initial_sidebar_state="expanded")

st.sidebar.header('User Input Parameters')
   
    # front end elements of the web page 
html_temp = """ 
    <div style ="background-color:light green;padding:13px"> 
    <h1 style ="color:orange;text-align:center;">NZ Hospital Demand Forecast APP</h1> 
    </div> 
    """
      
# display the front end aspect
st.markdown(html_temp, unsafe_allow_html = True) 
st.subheader('by Falalu Sani Barde, Manusree Banerjee, Sarah El Shatby, Toyin Ogunlade ')
      
# following lines create boxes in which user can enter data required to make prediction
page=st.sidebar.slider("Age",1,100)
psex = st.sidebar.radio("Select Gender: ", ('Male', 'Female'))
alpha_index = st.sidebar.selectbox('ICD Alphabetical Index', ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'))
code_num = st.sidebar.slider('ICD Numerical Index',0,99)
dhb = st.sidebar.selectbox("Select District: ", ('Auckland', 'Bay of Plenty', 'Canterbury', 'Capital and Coast', 'Counties Manukau', "Hawke's Bay", 'Hutt Valley', 'Lakes', 'Midcentral', 'Nelson Marlborough', 'Northland', 'South 3', 'South Canterbury', 'Southern', 'Tairawhiti', 'Taranaki', 'Waikato', 'Wairarapa', 'Waitemata', 'West Coast', 'Whanganui', 'Unknown'))
cols1,_ = st.columns((1,4)) # To make it narrower
format = 'MMM DD, YYYY'  # format output
start_date = dt.date(year=2021,month=1,day=1)  #  I need some range in the past
end_date = dt.date(year=2022,month=12,day=31)
max_days = end_date-start_date
date = st.sidebar.slider('Select admission date', min_value=start_date, value=end_date ,max_value=end_date, format=format)
## Sanity check
st.sidebar.table(pd.DataFrame([[start_date, date, end_date]],columns=['start','selected','end'],index=['date']))

if psex=='Female':
  sex=1
else:
  sex=0
if code_num<10:
  icdcode=alpha_index+'0'+str(code_num)
else:
  icdcode=alpha_index+str(code_num)
 
dist_dict = {'Auckland':1, 'Bay of Plenty':2, 'Canterbury':3, 'Capital and Coast':4, 'Counties Manukau':5, "Hawke's Bay":6, 'Hutt Valley':7, 'Lakes':8, 'Midcentral':9, 'Nelson Marlborough':10, 'Northland':11, 'South Canterbury':12, 'Southern':13, 'Tairawhiti':14, 'Taranaki':15, 'Unkown':16, 'Waikato':17, 'Wairarapa':18, 'Waitemata':19, 'West Coast':20, 'Whanganui':21}
# Accessing integer using roman key
district = dist_dict[dhb]

if st.button("PREDICT"):
  if icd10.exists(icdcode):
    code = icd10.find(icdcode)
    if icdcode=='A00' or icdcode=='B00':
      chap=1
    elif icdcode=='C00' or icdcode=='D00':
      chap=2
    elif icdcode=='E00':
      chap=4
    elif icdcode=='F00':
      chap=5
    elif icdcode=='G00':
      chap=6
    elif icdcode=='H00':
      chap=7
    elif icdcode=='I00':
      chap=9
    elif icdcode=='J00':
      chap=10
    elif icdcode=='K00':
      chap=11
    elif icdcode=='L00':
      chap=12
    elif icdcode=='M00':
      chap=13
    elif icdcode=='N00':
      chap=14
    elif icdcode=='O00':
      chap=15
    elif icdcode=='P00':
      chap=16
    elif icdcode=='Q00':
      chap=17
    elif icdcode=='R00':
      chap=18
    elif icdcode=='S00' or icdcode=='T00':
      chap=19
    elif icdcode=='U00':
      chap=22
    elif icdcode=='V00' or icdcode=='W00' or icdcode=='X00' or icdcode=='Y00':
      chap=20
    elif icdcode=='Z00':
      chap=21
    else:
      # Creating a Roman to Int Dictionary
      romtoint = {'I':1,'II':2,'III':3,'IV':4,'V':5,'VI':6,'VII':7,'VIII':8,'IX':9,'X':10,'XI':11,'XII':12,'XIII':13,'XIV':14,'XV':15,'XVI':16,'XVII':17,'XVIII':18,'XIX':19,'XX':20,'XXI':21,'XXII':22}
      # Accessing integer using roman key
      icd_chap=code.chapter
      chap=romtoint[icd_chap] 
    year=2021
    user_input=[[year,district,sex,page,chap]]
    #load the model from disk
    loaded_model = decompress_pickle('randforestreg.pbz2')
    mlos = loaded_model.predict(user_input) 
    st.write("Mean Length of Stay is ",np.round(mlos[0],0))
    dis_date=date+ timedelta(days=mlos[0])
    fdis_date = dis_date.strftime("%b %d, %Y")
    st.write("Predicted Discharge date is ",fdis_date)
  else:
    st.write("Entered ICD Code doesn't exist!!!")
  st.info("Don't forget to rate this app")
  
feedback = st.slider('How much would you rate this app?',min_value=0,max_value=5,step=1)
  
if feedback:
    st.header("Thank you for rating the app!")


st.subheader("About App")

st.info("This web app helps you to find approximate discharge date of a patient for the year 2021.")
st.info("Enter the required fields and click on the 'PREDICT' button to check your probable discharge date.")


st.caption("Caution: This is just a prediction and may not be exact.") 


Overwriting NZ_app.py


In [11]:
!ls

alldischarges_new.xlsx	NZ_app.py  randforestreg.pbz2  sample_data


In [12]:
!streamlit run NZ_app.py &>/dev/null&

In [13]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url

'http://c95e-35-226-181-254.ngrok.io'

In [10]:
ngrok.kill()